In [1]:
!pip install sense2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.0 MB/s eta 0:00:00


In [2]:
!pip install nltk

In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.3 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.6 MB/s eta 0:00:00:00:01


In [5]:
#import all the neccessary libraries
import warnings
warnings.filterwarnings("ignore")
import torch
from sense2vec import Sense2Vec
from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein
import pickle
import time
import os 

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
#getting the sentence transformer model and its tokenizer
#sentence transformer model often integrates the tokenizer within the model class
#we dont have to manage a separate tokenizer manually
if os.path.exists("sentence_transformer_model.pkl"):
    with open("sentence_transformer_model.pkl",'rb') as f:
        sentence_transformer_model = pickle.load(f)
    print("sentence transformer model found in the disc ---- model loaded successfully.")
else:
    print("sentence transformer model does not exist in the path specified ---- downloading the model from web.")
    start_time=time.time()
    sentence_transformer_model = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-v2")
    end_time=time.time()

    print("downloaded the sentence transformer in ",(end_time-start_time)/60," min, saving it to disc.")

    with open("sentence_transformer_model.pkl",'wb') as f:
        pickle.dump(sentence_transformer_model,f)

    print("saved the sentence transformer to disc.")

sentence transformer model does not exist in the path specified ---- downloading the model from web.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

downloaded the sentence transformer in  0.07816810210545858  min, saving it to disc.
saved the sentence transformer to disc.


In [7]:
def get_highest_similarity_score(wordlist,wrd):
  """
  takes the given word along with the wordlist and then gives out the max-score which is the levenshtein distance for the wrong answers
  because we need the options which are very different from one another but relating to the same context
  """
  score=[]
  for each in wordlist:
    score.append(Levenshtein.ratio(each.lower(),wrd.lower()))
  return max(score)

In [8]:
wordlist = ["bench", "bark", "bloom", "blank"]
target_word = "bank"

highest_score = get_highest_similarity_score(wordlist, target_word)

print("Highest Similarity Score:", highest_score)

Highest Similarity Score: 0.8888888888888888


In [9]:
def filter_same_sense_words(original,wordlist):
  
  """
  filter the words which are of same sense, where it takes the wordlist which has the sense of the word attached as the string along with the word itself.
  """
  filtered_words=[]
  base_sense =original.split('|')[1] 
  
  for eachword in wordlist:
    if eachword[0].split('|')[1] == base_sense:
      filtered_words.append(eachword[0].split('|')[0].replace("_", " ").title().strip())
  return filtered_words

In [10]:
original = "bank|financial_institution"
wordlist = [
    ["bank|financial_institution"],
    ["river_bank|location"],
    ["credit_union|financial_institution"],
    ["savings_bank|financial_institution"],
    ["shore|location"]
]

filtered = filter_same_sense_words(original, wordlist)

print("Filtered Words:", filtered)
#filtered words that have the same sense as the original word

Filtered Words: ['Bank', 'Credit Union', 'Savings Bank']


In [11]:
from sense2vec import Sense2Vec

#load the sense2vec model from the uploaded directory
s2v = Sense2Vec().from_disk('/kaggle/input/s2v-old/s2v_old')
print("sense2vec model loaded successfully.")

sense2vec model loaded successfully.


In [12]:
def sense2vec_get_words(word,s2v,topn,question):
    """
    This function takes the input word, sentence to vector model and top similar words and also the question
    Then it computes the sense of the given word
    then it gets the words which are of same sense but are most similar to the given word
    after that we we return the list of words which satisfy the above mentioned criteria
    """
    output = []
#     print ("word ",word)
    try:
      sense = s2v.get_best_sense(word, senses= ["NOUN", "PERSON","PRODUCT","LOC","ORG","EVENT","NORP","WORK OF ART","FAC","GPE","NUM","FACILITY"])
#       print(sense)
      most_similar = s2v.most_similar(sense, n=topn)
#       print (most_similar)
      output = filter_same_sense_words(sense,most_similar)
#       print ("Similar ",output)
    except:
      output =[]

    threshold = 0.6
    final=[word]
    checklist =question.split()
    for x in output:
      if get_highest_similarity_score(final,x)<threshold and x not in final and x not in checklist:
        final.append(x)
    
    return final[1:]

In [13]:
word = 'bank'
topn = 10
question = "What is the bank's interest rate on savings?"

similar_words = sense2vec_get_words(word, s2v, topn, question)

#output the similar words
print("Similar words to '{}': {}".format(word, similar_words))

Similar words to 'bank': ['Local Bank', 'Bank Account', 'Cash Deposits', 'Financial Institution', 'Cash', 'Different Bank']


In [14]:
"""
Maximal Marginal Relevance (MMR) algorithm to extract keywords or keyphrases from a document 
while balancing relevance and diversity
- vector representing the document's content
- matrix of word embeddings where each row corresponds to the embedding of a word in the vocabulary.
- list of words or keyphrases from which keywords will be selected
- number of keywords/keyphrases to extract from the document
"""
def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
    """
    The mmr function takes document and word embeddings, along with other parameters, 
    and uses the Maximal Marginal Relevance (MMR) algorithm to extract a specified number of keywords/keyphrases 
    from the document. The MMR algorithm balances the relevance of keywords with their diversity, 
    helping to select keywords that are both informative and distinct from each other.
    """

    #extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    #initialize candidates and already choose best keyword/keyphrase
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        #extract similarities within candidates and
        #between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        #calculate MMR
        mmr = (lambda_param) * candidate_similarities - (1-lambda_param) * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        #update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [15]:
def get_distractors (word,origsentence,sense2vecmodel,sentencemodel,top_n,lambdaval):
  """
  this function generates distractor words (answer choices) for a given target word in the context of a provided sentence. 
  It selects distractors based on their similarity to the target word's context and ensures that the target word itself 
  is not included among the distractors. This function is useful for creating multiple-choice questions or answer 
  options in natural language processing tasks.
  """
  distractors = sense2vec_get_words(word,sense2vecmodel,top_n,origsentence)
  #print ("distractors ",distractors)
  if len(distractors) ==0:
    return distractors
  distractors_new = [word.capitalize()]
  distractors_new.extend(distractors)
  # print ("distractors_new .. ",distractors_new)

  embedding_sentence = origsentence+ " "+word.capitalize()
  # embedding_sentence = word
  keyword_embedding = sentencemodel.encode([embedding_sentence])
  distractor_embeddings = sentencemodel.encode(distractors_new)

  # filtered_keywords = mmr(keyword_embedding, distractor_embeddings,distractors,4,0.7)
  max_keywords = min(len(distractors_new),5)
  filtered_keywords = mmr(keyword_embedding, distractor_embeddings,distractors_new,max_keywords,lambdaval)
  # filtered_keywords = filtered_keywords[1:]
  final = [word.capitalize()]
  for wrd in filtered_keywords:
    if wrd.lower() !=word.lower():
      final.append(wrd.capitalize())
  final = final[1:]
  return final

In [16]:
#target word and original sentence
word = "dog"
origsentence = "The dog barked loudly."

top_n = 5  #number of top similar words
lambdaval = 0.5  #balance parameter for MMR

distractors = get_distractors(word, origsentence, s2v, sentence_transformer_model, top_n, lambdaval)

# Output the distractors
print("Distractors for '{}': {}".format(word, distractors))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Distractors for 'dog': ['Kitten', 'Puppy', 'Cat']
